In [ ]:
from datetime import datetime
import speech_recognition as sr
import pyttsx3
import webbrowser
import requests

In [ ]:
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voices', voices[1].id)

def say(text, rate = 120):
    engine.setProperty('rate', rate)
    print(text)
    engine.say(text)
    engine.runAndWait()

In [ ]:
def takeCommand():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.pause_threshold = 0.6
        audio = r.listen(source)
        try:
            print('Recoginizing...')
            query = r.recognize_google(audio, language='en-us')
            print(f'User said: {query}.')
            return query
        except Exception as e:
            return 'Some Error Occured.'

In [ ]:
def time():
    hour = datetime.now().strftime('%#I')
    min = datetime.now().strftime('%M')
    if min == 00:
        return f"The time is {hour} O'Clock"
    elif min == 30:
        return f"The time is half past {hour}"
    elif min == 15:
        return f"The time is quater past {hour}"
    elif min == 45:
        return f"The time is quater to {hour+1}"
    else:
        return f"The time is {hour}:{min}"

In [ ]:
def kelvin_to_celsius_farenheit(kelvin):
    celsius = kelvin - 273.15
    farenheit = celsius * (9/5) +32
    return celsius, farenheit

def weather(query):
    BASE_URL = 'http://api.openweathermap.org/data/2.5/weather?'
    API_KEY = 'a00090c8702742cab92391097c57b42d'
    if query.split(' ')[-1] == 'now':
        CITY = query.split(' ')[-2]
    else:
        CITY = query.split(' ')[-1]

    url = BASE_URL + 'appid=' + API_KEY + '&q=' + CITY
    response = requests.get(url).json()
    print(response)

    temp_kelvin = response['main']['temp']
    temp_celsius, temp_farenheit = kelvin_to_celsius_farenheit(temp_kelvin)
    feels_like_kelvin = response['main']['feels_like']
    feels_like_celsius, feels_like_farenheit = kelvin_to_celsius_farenheit(feels_like_kelvin)
    wind_speed = response['wind']['speed']
    humidity = response['main']['humidity']
    description = response['weather'][0]['description']
    sunrise_time = datetime.utcfromtimestamp(response['sys']['sunrise'] + response['timezone'])
    sunset_time = datetime.utcfromtimestamp(response['sys']['sunset'] + response['timezone'])
    
    if 'temperature' in query.lower():
        return f'The temperature is {int(temp_celsius)} degrees celsius'
    elif 'temperature in farenheit' in query.lower():
        return f'The temperature is {int(temp_farenheit)} degrees farenheit'
    elif 'temperature in kelvin' in query.lower():
        return f'The temperature is {int(temp_kelvin)} kelvin'
    elif 'temperature feels like' in query.lower():
        return f'The temperature feels like {int(feels_like_celsius)} degrees celsius'
    elif 'temperature feels like in farenheit' in query.lower():
        return f'The temperature feels like {int(feels_like_farenheit)} degrees farenheit'
    elif 'temperature feels like in kelvin' in query.lower():
        return f'The temperature feels like {int(feels_like_kelvin)} kelvin'
    elif 'humidity' in query.lower():
        return f'The humidity is {humidity} percent'
    elif 'sunrise time' in query.lower():
        return f'The sunrise time is {sunrise_time.split(" ")[1]}'
    elif 'sunset time' in query.lower():
        return f'The sunset time is {sunset_time.split(" ")[1]}'
    elif 'wind speed' in query.lower():
        return f'The wind speed is {wind_speed} kilometers'
    elif 'weather' in query.lower():
        return f'It is {description}'


In [ ]:
if __name__ == '__main__':
    say('Hello, I am Jarvis AI')
    while True:
        print('Listening...')
        query = takeCommand()
        sites = [['youtube', 'https://www.youtube.com'],
                 ['linkedin', 'https://www.linkedin.com'],
                 ['facebook', 'https://www.facebook.com'],
                 ['wikipedia', 'https://www.wikipedia.com'],
                 ['google', 'https://www.google.com'],
                 ['twitter', 'https://www.twitter.com'],
                 ['instagram', 'https://www.instagram.com'],
                 ['snapchat', 'https://www.snapchat.com'],
                 ['quora', 'https://www.quora.com']]
        for site in sites:
            if f'Open {site[0]}' in query.lower():
                say(f'Opening {site[0]}')
                webbrowser.open(site[1])
        if 'the time' in query.lower():
            say(time())
        elif 'temperature' or 'humidity' or 'wind speed' or 'feels like' or 'weather' or 'sunset time' or 'sunrise time' in query.lower():
            say(weather(query))
        elif 'exit' in query.lower():
            say('Shutting down')
            exit()